Much of this workflow is built apon the work done by Stephan Rasp for his CBRAIN Github repository.  However, some modifications have been made to account for changes in data, model design and different post-processing tests

# Preprocessing

The bash script to preprocess the data is found under /Bash_AScripts/Preprocessor.sh - an example for a 9 years training data/1 year validation data (selectively sampled every 10 days) is shown.  Corresponding files to help in the preprocessing are

- preprocess_RG.py
- SPCAM5.yml
- shuffle_ds.py

Once the bash script has been run, a preprocessed and shuffled training and validation dataset will be ready for use by the deep neural network.  The input and outrput variables of interest can be specified in SPCAM5.yml file. 

# Training

The file Deep_Training.py shows how we used manual tuning as a comparison to the SHERPA automated hyperparameter tuning packaged we used to get our best DNN. Key corresponding files include

- losses.py
- utils.py
- models.py
- imports.py
- hyai_hybi.pkl
- data_generator.py

The file produces a .h5 model.

# Getting DNN Predictions

The script Model_Predictions.py takes the .h5 model and the corresponding validation dataset to get out model predictions in a .nc file.  This file can now be used for post-processing analysis

# Post Processing

# Table 1

The values that compose Table 1 in the Manuscript are generated from the scripts:

- SPCAM5_R2.ipynb

In which data:

   - Preprocessed_Data/SPCAM3_6_Months/full_physics_essentials_valid_month02_targets.nc
- Models/SPCAM3_6_Months.nc

Corresponds to SPCAM3 Values,

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

Corresponds To Sherpa Values,

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Spaced_Fig_4_7_Years.nc


Corresponds to Manually Tuned Values

And then 

- SPCAM5_R2.ipynb

and 

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

Correspond to to the daily values

#  Figure 1

The values for Figure 1 are generated by Lat_Pressure_Timestep_R2_Heating.ipynb and Lat_Pressure_Timestep_R2_Moistening.ipynb, placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb. 

In Lat_Pressure_Timestep_R2_Heating.ipynb, Lat_Pressure_Timestep_R2_Moistening.ipynb, the file paths for the SPCAM3 results (panels a and b) customize the file paths to:

- Preprocessed_Data/SPCAM3_6_Months/full_physics_essentials_valid_month02_targets.nc
- Models/SPCAM3_6_Months.nc

The data (from Rasp et al.) above reference the target data and the DNN (designed based on Rasp et al.) predictions respectively on SPCAM3 data.  Ensure x = 128 and y = 64 to match SPCAM3 spatial resolution. The script takes care of the rest.

In Lat_Pressure_Timestep_R2_Heating.ipynb, Lat_Pressure_Timestep_R2_Moistening.ipynb, the file paths for the SPCAM5 results (panels c and d) customize the file paths to:

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

The paths above reference the validation dataset (10 year) of SPCAM5 data and the best SHERPA DNN we have. Ensure x = 144 and y = 96 to match SPCAM5 spatial resolution. The script takes care of the rest.


# Figure 2

The values for Figure 2 are generated by Lat_Pressure_Timestep_R2_Heating.ipynb, placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb

In Lat_Pressure_Timestep_R2_Heating.ipynb, the file paths for the manually tuned SPCAM5 results (panel a) customize the file paths to:

- Preprocessed_Data/SPCAM3_6_Months/full_physics_essentials_valid_month02_targets.nc
- Models/SPCAM3_6_Months.nc


In Lat_Pressure_Timestep_R2_Heating.ipynb, the file paths for the manually tuned SPCAM5 results (panel b) customize the file paths to:

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Spaced_Fig_4_7_Years.nc

In Lat_Pressure_Timestep_R2_Heating.ipynb, the file paths for the SHERPA SPCAM5 results (panel c) customize the file paths to:

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

The paths above reference the validation dataset (10 year) of SPCAM5 data and the best SHERPA DNN we have. Ensure x = 144 and y = 96 to match SPCAM5 spatial resolution. The script takes care of the rest.


# Figure 3

The values for Figure 3 are generated by Lat_Pressure_Timestep_R2_Heating.ipynb and Lat_Pressure_Timestep_R2_Moistening.ipynb, Lat_Pressure_Daily_R2_Heating.ipynb, Lat_Pressure_Daily_R2_Moistening.ipynb, placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb. 

In Lat_Pressure_Timestep_R2_Heating.ipynb, Lat_Pressure_Timestep_R2_Moistening.ipynb, Lat_Pressure_Daily_R2_Heating.ipynb, Lat_Pressure_Daily_R2_Moistening.ipynb, the file paths for the SPCAM5 results (panels a-d) customize the file paths to:

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

The paths above reference the validation dataset (10 year) of SPCAM5 data and the best SHERPA DNN we have. Ensure x = 144 and y = 96 to match SPCAM5 spatial resolution. The script takes care of the rest.

# Figure 4

The values for Figure 4 are generated in SPCAM5_R2.ipynb placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb

In SPCAM5_R2.ipynb, the file paths for the SHERPA SPCAM5 results (panels b) customize the file paths to:

- Preprocessed_Data/9_Years_Spaced/full_physics_essentials_valid_month02_targets.nc
- Models/Jordan_Best.nc

The paths above reference the validation dataset (10 year) of SPCAM5 data and the best SHERPA DNN we have. Ensure x = 144 and y = 96 to match SPCAM5 spatial resolution. The script takes care of the rest.

# Figure 5

The values for Figure 5 are generated from CAM_Power_Spectra.ipynb, Paper_PSD_Heating.ipynb, Paper_PSD_Moistening.ipynb.  The compressed data generated in the figures above is opened and displayed via Submitted_Figures.ipynb.  File paths to target data and DNN predictions are provided in the script.

# Figure 6

The values for Figure 6 A are generated in SPCAM5_R2.ipynb placed into a .npy file, and displayed in final form in Final_Paper_Figures.ipynb

In SPCAM5_R2.ipynb, the file paths for the SHERPA SPCAM5 results (panels b) customize the file paths to:

- Preprocessed_Data/One_Month_Apr/full_physics_essentials_valid_month02_targets.nc
- Models/Good_Apr.nc

The paths above reference the validation dataset looking omnly at April - not selectively sampled and the best SHERPA DNN we have. Ensure x = 144 and y = 96 to match SPCAM5 spatial resolution. The script takes care of the rest.

The values for Figure 6 B are generated in Paper_Time_Series_Signal_Analysis.ipynb placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb

# Figure 7

The values for Figure 7 are generated in Paper_Time_Series_Signal_Analysis.ipynb placed into a .npy file, and displayed in final form in Submitted_Figures.ipynb

# Figure 8

The values for figure 8 are developed through the Improved_Diurnal_Heating_Moistening_Cycle.ipynb script and displayed in Submitted_Figures.ipynb.  The file paths are hardcoded in.

# Figure 9

The values for Figure 9 are developed in New_LST_Map.ipynb and displayed in Submitted_Figures.ipynb. The file paths are hardcoded in.

# Figure 10

The values in Figure 10 are developsed in Annual_Precip_Figures.ipynb and displayed in Submitted_Figures.ipynb.  The file paths are hardcoded in.

# Figure 11

Figure 11 is taken from the results of Galen Yacalis's Master's thesis.  Citations are in the manuscript

# Figure 12

The values for Figure 12 are generated by Error_and_Variance.ipynb and displayed in final form in Submitted_Figures.ipynb. The paths are hard coded in. 


# Figure 13

The values for Figure 13, panels a, b, and c are generated by Error_and_Variance.ipynb and displayed in final form in Submitted_Figures.ipynb. The paths are hard coded in. For panels e, d, and f, they are taken from Han et al 2020

# Animations

All embedded animations are generated in Animation_For_Paper.ipynb and the file paths are hard coded in.